- selective select 문제 : 크기가 다른 문제

intersection of Union(IOU) : 이미지에서 실제 부분과 예측 부분 교집합/전체집합
- localization을 높이기 위함.

### history

2012
    - CL : AlexNet, Selective Search(CTSF)
    - DE : R-CNN
2013
    - CL : ZFNet
2014
    - CL : Googlenet, VGG
    - DE : SPPNet
2015
    - CL : Resnet
    - DE : Fast R-CNN

- Bags of words
    - histogram(어떤 특징이 얼마나 있는지)
    - Spatial pyramid 구조에서 영역별 histogram구함
    
- Sppnet
    - 처음에는 svm 나중에는 fully connected 사용
    
    
---

## Fast R-CNN
- **ROI Layer**
    - 학습가능하도록 다른 사이즈를 같은 정사각형으로 만듬 -> pooling 기법
    - pyramid 방법이 아닌 layer 하나로 pooling
- input size를 hierarchy하게 만듬
- VGG 사용(기존 Alexnet)
- localization, prediction model 하나로 만듬 -> end to end


### multi-task loss
- softmax classifier + b-box regressors 합쳐서 동시에 loss 최소화
- end-to-end model
- one network traind in one stage
- Log loss + smooth L1 loss 최소화

<br>

- conclusion
![fast_rcnn.PNG](fast_rcnn.PNG)

### ROI Pooling
- selective search 에서 하나가 roi라고 할 수 있음.

https://github.com/yhenon/keras-spp/blob/master/spp/RoiPooling.py

- 위 기반 구현

### Layer customizing
- build : 초기화 값 실행 이후에는 안 불림, 초기화 체크
- call 만들어져 있으면 어떻게 해야하는지 알려줌

In [1]:
import tensorflow as tf

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [7]:
from tensorflow.keras import backend as K

In [4]:
class ROILayer(tf.keras.layers.Layer):
    def __init__(self, pool_list, num_rois,**kwags): # 기본적인 시작
        super().__init__(**kwags)
        self.poolist = pool_list # pooling regions 갯수 리스트
        self.num_rois = num_rois  # 채널 당 output수
        self.num_outputs_per_channer = sum(i*i for i in pool_list)
        # 2개 -> output 4개
        
    def build(self, input_shape): # input 받았을 때 모양
  #  Channel Last
  # 채널수 초기화(리턴 없음)  : 데이터 구조  (0HWC) 

        self.nb_channels = input_shape[0][3]  # (0,h,w,c)
    def call(self, x): # 여기가 중요
        # 어떻게 쪼갤 것인가
        # image 당 selective 몇개인지
        # convolution 나온 것을 ROI Layer에 넣어 (7,7)로 나누기
        # pyramid 기법 안써서 빠른 속도
        
        img = x[0]
        rois = x[1]
        for i in range(self.num_rois): #
            x = rois[0,i,0]
            y = rois[0,i,1]
            w = rois[0,i,2]
            h = rois[0,i,3]
            
            # pool_list 기반 분할
            row_leng = [h/j for j in self.pool_list]
            cow_leng = [w/j for j in self.pool_list]
            
            # max pooling
            # layer 덮어쓰면 작은 kernel로도 성능 어느 정도 낼 수 있음.
            # VGGNet kernel size 3x3 여러개 하면 더 큰 커널사용한것과 동일한 효과냄 
            # 한번에 7x7 pooling -> 속도 빠르다
            
            for i in poll_region:
                for j in i :
                    
                    x1 = x + ix * col_length[pool_num]
                    x2 = x1 + col_length[pool_num]
                    y1 = y + jy * row_length[pool_num]
                    y2 = y1 + row_length[pool_num]

                    x1 = K.cast(K.round(x1), 'int32')
                    x2 = K.cast(K.round(x2), 'int32')
                    y1 = K.cast(K.round(y1), 'int32')
                    y2 = K.cast(K.round(y2), 'int32')

                    new_shape = [input_shape[0], y2 - y1,
                                         x2 - x1, input_shape[3]]
                    x_crop = img[:, y1:y2, x1:x2, :]
                    xm = K.reshape(x_crop, new_shape)
                            
                    pool_val = K.max()
                    self.output.append(pool_val)
                    
            
#         print(x)  x가 뭔지 debugging
    
    def compute_output_shape(self, input_shape): # output을 어떻게 내주는가
        # 확인; (None,num_rois, 총 output 갯수 )
        return None, self.num_rois,self.nb_channels*self.num_output_per_channel
    
    def get_config(self): # 구성 확인
        original = super().get_config()
        my_config = {'pool_list': self.pool_list, 'num_rois' : self.num_rois}
        original.update(my_config)
        return original

In [ ]:
vgg=tf.keras.applications.VGG16()
vgg.get_config()

In [5]:
r1 = ROILayer([1,2,4],3)
r1.get_config()

In [6]:
# dict 추가 , mutable 방식
x = {'a':1,'b':2}
y = {'c':3}
x.update(y) # return none
x

{'a': 1, 'b': 2, 'c': 3}

2016
    - DE : Faster R-CNN
    
#### Fast R-CNN problen
- 4가지 요소 이외 모르는 영역을 묶는 것이 있을 거란 생각을 하게 됨
- selective search자체가 속도가 느림.
- end to end도 아님
    
## Faster R-CNN
- selective search algorithm 포기 -> 학습을 통한 것이 아니었고 학습
- 정확도는 어느정도 왔고 속도를 높이는데에 초점
- feature maps - SPP/ RoI pooling layers 로 크기 맞춤